In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

from scipy.io.matlab import loadmat
import string

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# Load Data

In [ ]:
def load_idx3_ubyte(file_path='../data/t10k-images.idx3-ubyte'):

    with open(file_path, 'rb') as f:
        
        magic_number = int.from_bytes(f.read(4), 'big')
        num_items = int.from_bytes(f.read(4), 'big')

        if magic_number == 2051:  # Fichier d'images
            num_rows = int.from_bytes(f.read(4), 'big')
            num_cols = int.from_bytes(f.read(4), 'big')
            data = np.frombuffer(f.read(), dtype=np.uint8)
            data = data.reshape(num_items, num_rows, num_cols)
            
        elif magic_number == 2049:  # Fichier d'étiquettes
            data = np.frombuffer(f.read(), dtype=np.uint8)
            
        else:
            raise ValueError("Format de fichier .idx3-ubyte non reconnu")
            
    return data


def lire_alpha_digit(file_path='../data/binaryalphadigs.mat',caractere=['0']):
    
    assert caractere, "List not empty"

    elements = [str(i) for i in range(10)] + list(string.ascii_uppercase)
    
    index_caractere_list = [elements.index(c) for c in caractere if c in elements]
    if not index_caractere_list:
        raise ValueError("One or many caracters are not recognized.")

    data = loadmat(file_path)
    
    size_img = data['dat'][0][0].shape
    nb_pixel = size_img[0]*size_img[1]
    

    X = data['dat'][np.array(index_caractere_list)]
    X = np.concatenate(X)
    X = np.concatenate(X).reshape((X.shape[0],nb_pixel))

    return X, size_img

In [ ]:
def plot_images(X, size_img):
    num_images = len(X)
    
    # Calculate the number of rows and columns for the subplots
    cols = np.ceil(np.sqrt(num_images)) 
    rows = np.ceil(num_images / cols)     
    
    fig, axes = plt.subplots(int(rows), int(cols), figsize=(cols * 2, rows * 2))
    axes = axes.flatten() 
    
    for i, image in enumerate(X):
        image = image.reshape(size_img)
        axes[i].imshow(image, cmap='gray')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout() 
    plt.show()

In [ ]:
X, size_img = lire_alpha_digit('../data/binaryalphadigs.mat',caractere=['A'])
plot_images(X, size_img)

# RBM

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
class RBM:
    def __init__(self, p, q):
        self.a = np.zeros(p)
        self.b = np.zeros(q)
        self.W = np.random.normal(size=(p, q)) * np.sqrt(0.01)

    def entree_sortie_RBM(self, V):
        return sigmoid(V @ self.W + self.b) 

    def sortie_entree_RBM(self, H):
        return sigmoid(H @ self.W.T + self.a)

    def train_RBM(self, X, learning_rate, len_batch, n_epochs,verbose=1):
        p, q = self.W.shape

        n_samples = X.shape[0]
    
        for epoch in range(n_epochs):
            
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            
            for ith_batch in range(0, n_samples, len_batch):
                
                X_batch = X_shuffled[ith_batch:ith_batch + len_batch]
                m = X_batch.shape[0]

                #Contrastive-Divergence-1 algorithm to estimate the gradient
                V0 = X_batch.copy()
                
                pH_V0 = self.entree_sortie_RBM(V0)
                # draw from pH_V0
                H0 = (np.random.rand(m, q) < pH_V0) * 1
                
                pV_H0 = self.sortie_entree_RBM(H0)
                # draw from pV_H0
                V1 = (np.random.rand(m, p) < pV_H0) * 1
                
                pH_V1 = self.entree_sortie_RBM(V1)

                grad_a = np.sum(V0 - V1, axis=0)
                grad_b = np.sum(pH_V0 - pH_V1, axis=0)
                grad_W = V0.T @ pH_V0 - V1.T @ pH_V1

                self.a += learning_rate * grad_a
                self.b += learning_rate * grad_b
                self.W += learning_rate * grad_W

            # Reconstruction's loss
            H = self.entree_sortie_RBM(X_shuffled)
            X_rec = self.sortie_entree_RBM(H)
            loss = np.mean((X_shuffled - X_rec) ** 2) #quadratic norm

            if epoch % 10 == 0 and verbose: # verbose for progression bar
                print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}")
    
    def generer_image_RBM(self, nb_images, nb_iter):
        p, q = self.W.shape
        images = []
        
        for i in range(nb_images):  # Gibbs
            v = (np.random.rand(p) < 0.5) * 1
            for j in range(nb_iter):
                h = (np.random.rand(q) < self.entree_sortie_RBM(v)) * 1
                v = (np.random.rand(p) < self.sortie_entree_RBM(h)) * 1
            images.append(v)
            
        return images

In [ ]:
X, size_img = lire_alpha_digit('../data/binaryalphadigs.mat', caractere=['A'])

nb_features = size_img[0] * size_img[1]
p, q = nb_features, 100
rbm = RBM(p, q)  # Instance of RBM

rbm.train_RBM(X, learning_rate=10 ** (-2), len_batch=10, n_epochs=1000, verbose=0)

generated_images = rbm.generer_image_RBM(nb_images=10, nb_iter=200)
plot_images(generated_images, size_img)

# DBN

In [ ]:
class DBN:
    def __init__(self, dbn_size):
        """
        :param dbn_size: List [] of numbers of neurons per layer.
        """
        self.dbn_size = dbn_size
        self.rbms = []  # List to store RBMs

        # Initialize RBMs for each pair of consecutive layers
        for l in range(len(dbn_size) - 1):
            p = dbn_size[l]
            q = dbn_size[l + 1]
            rbm = RBM(p, q)
            self.rbms.append(rbm)

    def train_DBN(self, X, learning_rate, len_batch, n_epochs, verbose=1):
        
        tmp = X.copy()
        
        #Greedy layer wise procedure
        for l in range(len(self.dbn_size) - 1):
            if verbose:
                print(f"Train RBM {l+1}/{len(self.dbn_size)-1}\t") 
            self.rbms[l].train_RBM(tmp, learning_rate, len_batch, n_epochs,verbose)
            tmp = self.rbms[l].entree_sortie_RBM(tmp)


    def generer_image_DBN(self, nb_images, nb_iter):
        images = []

        for i in range(nb_images):

            #Gibbs sur la derniere couche cachée
            top_rbm = self.rbms[-1]
            p, q = top_rbm.W.shape
            
            # Initialisation aléatoire de la couche visible
            v = (np.random.rand(p) < 0.5) * 1

            for _ in range(nb_iter):
                h = (np.random.rand(q) < top_rbm.entree_sortie_RBM(v)) * 1
                v = (np.random.rand(p) < top_rbm.sortie_entree_RBM(h)) * 1

            # Propagation vers l'arrière (caché -> visible)
            h = v
            for rbm in reversed(self.rbms[:-1]):
                p, q = rbm.W.shape
                h = (np.random.rand(p) < rbm.sortie_entree_RBM(h)) * 1

            images.append(h)

        return images

In [ ]:
X, size_img = lire_alpha_digit('../data/binaryalphadigs.mat', caractere=['A'])

dbn_size = [320, 200, 200]
dbn = DBN(dbn_size)  # Instance of DBN
dbn.train_DBN(X, learning_rate=1e-2, len_batch=100, n_epochs=100, verbose=0)

generated_images = dbn.generer_image_DBN(nb_images=10, nb_iter=200)
plot_images(generated_images, size_img)

# DNN

In [ ]:
def process_images_MNIST(file_path='../data/train-images-idx3-ubyte'):
    images = load_idx3_ubyte(file_path)
    
    size_img = images[0].shape
    images = images.reshape((images.shape[0],size_img[0]*size_img[1]))
    
    images = np.round(images/255) # binary MNIST
    
    return images, size_img

def one_hot_encoding(labels, nb_classes):
    """
    Converts a label vector into a one-hot encoded matrix.
    """
    one_hot = np.zeros((len(labels), nb_classes))
    one_hot[np.arange(len(labels)), labels] = 1
    return one_hot

def calcul_softmax(X):
    """
    Args:
        X (np.ndarray)
    """
    assert isinstance(X, np.ndarray), "Please use array."
    
    if X.ndim == 1:
        return np.exp(X)/np.sum(np.exp(X))
    elif X.ndim == 2:
        return np.exp(X)/np.sum(np.exp(X), axis=1, keepdims=True)
    else:    
        raise ValueError("1 or 2 dimensional array.")

In [ ]:
class DNN:
    def __init__(self, network_size):
        """
        Args:
            network_size: size of the network including the layer for classification
        """
        self.dbn = DBN(network_size[:-1])
        nb_classes = network_size[-1]
        self.W_l = np.random.randn(network_size[-2], nb_classes) * np.sqrt(0.01)  # Weights for classification layer
        self.b_l = np.zeros(nb_classes)  # Bias for classification layer


    def pretrain_DNN(self, X, learning_rate, len_batch, n_epochs, verbose=1):
        self.dbn.train_DBN(X, learning_rate, len_batch, n_epochs, verbose)

    def entree_sortie_reseau(self, X):
        """
        Store and return inputs + the outputs of each layer.
        """
        
        outputs = [X]
        
        h = X.copy()
        for rbm in self.dbn.rbms:
            h = rbm.entree_sortie_RBM(h)
            outputs.append(h)

        y_hat = calcul_softmax(h @ self.W_l + self.b_l)
        outputs.append(y_hat)
    
        return outputs

    def retropropagation(self, X, labels, learning_rate, len_batch, n_epochs, verbose=1):
        """
        Perform backpropagation to fine-tune the DBN using labels.
        Args:
            labels (np.ndarray): one-hot encoded labels.
        """
        
        n_samples = X.shape[0]
    
        for epoch in range(n_epochs):
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = labels[indices]
    
            for ith_batch in range(0, n_samples, len_batch):

                X_batch = X_shuffled[ith_batch:ith_batch + len_batch]
                y_batch = y_shuffled[ith_batch:ith_batch + len_batch]

                m = X_batch.shape[0]
    
                # Forward pass
                outputs = self.entree_sortie_reseau(X_batch)
                y_hat = outputs[-1]
                
                # Classification layer gradients
                c = y_hat - y_batch

                grad_w = outputs[-2].T @ c / m 
                grad_b = np.mean(c, axis=0) 
                
                self.W_l -= learning_rate * grad_w
                self.b_l -= learning_rate * grad_b

                # Backward pass through hidden layers
                weight = self.W_l.copy()
                for l in range(len(outputs)-2, 0, -1):
                    x = outputs[l]
                    c = (c @ weight.T) * (x * (1 - x))

                    x_prev = outputs[l-1]
                    grad_w = x_prev.T @ c / m
                    grad_b = np.mean(c, axis=0)

                    rbm = self.dbn.rbms[l-1]
                    rbm.W -= learning_rate * grad_w
                    rbm.b -= learning_rate * grad_b
                    weight = rbm.W

            # Reconstruction of the Cross-entropy loss
            outputs = self.entree_sortie_reseau(X_shuffled)
            y_hat = outputs[-1]

            # Cross-entropy loss
            loss = -np.sum(y_shuffled * np.log(y_hat)) / n_samples
            
            if epoch % 10 == 0 and verbose:             
                print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}")
                
    def test_DNN(self, X, labels):
        """
        Compute the error rate.
        
        Args:
            labels (np.ndarray).
        """
        
        # Retrieve estimated label
        outputs = self.entree_sortie_reseau(X)
        y_hat = outputs[-1]
        label_estimated = np.argmax(y_hat, axis=1)

        errors = (labels != label_estimated).astype(int)
        return np.mean(errors)

In [ ]:
nb_classes = 10

train_images, train_size_img = process_images_MNIST('../data/train-images-idx3-ubyte')
train_labels = load_idx3_ubyte('../data/train-labels-idx1-ubyte')
encoded_train_labels = one_hot_encoding(train_labels, nb_classes)

test_images, test_size_img = process_images_MNIST('../data/t10k-images-idx3-ubyte')
test_labels = load_idx3_ubyte('../data/t10k-labels-idx1-ubyte')

nb_features = train_size_img[0]*train_size_img[1]

In [ ]:
dnn_pretrained = DNN(network_size=[nb_features, 600, 600, 600, nb_classes])

print("----------------------------------------------------- Pre-training -----------------------------------------------------")
dnn_pretrained.pretrain_DNN(train_images,learning_rate=1e-2, len_batch=100, n_epochs=100, verbose=0)
print("-------------------------------------------------- Back-Propragation ---------------------------------------------------")
dnn_pretrained.retropropagation(train_images, encoded_train_labels, learning_rate=1e-2, len_batch=100, n_epochs=200, verbose=0)
print("----------------------------------------------------- Error Rate -------------------------------------------------------")
error_rate = dnn_pretrained.test_DNN(test_images, test_labels)
print(f"Error rate: {error_rate*100}%")

In [ ]:
dnn = DNN(network_size=[nb_features, 200, 200, nb_classes])

print("-------------------------------------------------- Back-Propragation ---------------------------------------------------")
dnn.retropropagation(train_images, encoded_train_labels, learning_rate=1e-2, len_batch=100, n_epochs=200, verbose=0)
print("----------------------------------------------------- Error Rate -------------------------------------------------------")
error_rate = dnn.test_DNN(test_images, test_labels)
print(f"Error rate: {error_rate*100}%")

# Analysis

In [ ]:
nb_classes = 10

train_images, train_size_img = process_images_MNIST('../data/train-images-idx3-ubyte')
train_labels = load_idx3_ubyte('../data/train-labels-idx1-ubyte')
encoded_train_labels = one_hot_encoding(train_labels, nb_classes)

test_images, test_size_img = process_images_MNIST('../data/t10k-images-idx3-ubyte')
test_labels = load_idx3_ubyte('../data/t10k-labels-idx1-ubyte')

nb_features = train_size_img[0]*train_size_img[1]

In [ ]:
def train_dnn(network_size, train_images, encoded_train_labels, pretrain=True):
    dnn = DNN(network_size=network_size)
    if pretrain:
        dnn.pretrain_DNN(train_images, learning_rate=1e-2, len_batch=10, n_epochs=10, verbose=0)
    dnn.retropropagation(train_images, encoded_train_labels, learning_rate=1e-2, len_batch=10, n_epochs=20, verbose=0)
    return dnn

In [ ]:
to_study = pd.DataFrame(columns=['Layers', 'Neurons', 'Train data', 'Error Train', 'Error Test'])

nb_layers_dbn = np.arange(1,6)
tmp = pd.DataFrame(data={'Layers': nb_layers_dbn, 'Neurons': [200]*len(nb_layers_dbn), 'Train data': [len(train_images)]*len(nb_layers_dbn)})
to_study = pd.concat([to_study, tmp], axis=0, ignore_index=True)

nb_neurons_dbn = np.arange(1,8)*100
tmp = pd.DataFrame(data={'Layers': [2]*len(nb_neurons_dbn), 'Neurons': nb_neurons_dbn, 'Train data': [len(train_images)]*len(nb_neurons_dbn)})
to_study = pd.concat([to_study, tmp], axis=0, ignore_index=True)

nb_train_data = [1000, 3000, 7000, 10000, 30000, 60000]
tmp = pd.DataFrame(data={'Layers': [2]*len(nb_train_data), 'Neurons': [200]*len(nb_train_data), 'Train data': nb_train_data})
to_study = pd.concat([to_study, tmp], axis=0, ignore_index=True)

to_study.drop_duplicates(subset=["Layers", "Neurons", "Train data"], ignore_index=True, inplace=True)
to_study.sort_values(["Train data", "Layers", "Neurons"], ignore_index=True, inplace = True)

to_study["Pre-trained"] = False

to_study = pd.concat([to_study, to_study], ignore_index=True)
to_study.loc[len(to_study)//2:, "Pre-trained"] = True

In [ ]:
for i, row in to_study.iterrows():
    nb_layers_dbn = row["Layers"]
    nb_neurons_dbn = row["Neurons"]
    nb_data = row["Train data"]
    pretrain = row["Pre-trained"]

    network_size = [nb_features] + [nb_neurons_dbn for i in range(nb_layers_dbn)] + [nb_classes]

    trained_dnn = train_dnn(network_size, train_images[:nb_data], encoded_train_labels[:nb_data], pretrain=pretrain)
    
    error_rate_train = trained_dnn.test_DNN(train_images, train_labels)
    error_rate_test = trained_dnn.test_DNN(test_images, test_labels)
    
    to_study.iloc[i, 3] = error_rate_train*100
    to_study.iloc[i, 4] = error_rate_test*100

    if i%5 == 0:    
        print(f"Run - {i}/{len(to_study)-1}")

In [ ]:
pretrained = to_study.query("`Pre-trained` == True and Neurons == 200 and `Train data` == 60000")
without_pretrained = to_study.query("`Pre-trained` == False and Neurons == 200 and `Train data` == 60000")

nb_layers_dbn = pretrained["Layers"].values

e_pretrained_dnn_train = pretrained["Error Train"].values
e_pretrained_dnn_test = pretrained["Error Test"].values
e_dnn_train = without_pretrained["Error Train"].values
e_dnn_test = without_pretrained["Error Test"].values

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].plot(nb_layers_dbn, e_pretrained_dnn_train, label='Pre-trained DNN', marker='o', color='blue')
axs[0].set_xlabel('Number of Hidden Layers in DBN')
axs[0].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[0].tick_params(axis='y', labelcolor='blue')
axs[0].set_title('Train')
axs[0].grid(True)

axs_twin_train = axs[0].twinx()
axs_twin_train.plot(nb_layers_dbn, e_dnn_train, label='Standard DNN', marker='o', color='orange')
axs_twin_train.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_train.tick_params(axis='y', labelcolor='orange')

axs[1].plot(nb_layers_dbn, e_pretrained_dnn_test, label='Pre-trained DNN', marker='o', color='blue')
axs[1].set_xlabel('Number of Hidden Layers in DBN')
axs[1].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[1].tick_params(axis='y', labelcolor='blue')
axs[1].set_title('Test')
axs[1].grid(True)

axs_twin_test = axs[1].twinx()  
axs_twin_test.plot(nb_layers_dbn, e_dnn_test, label='Standard DNN', marker='o', color='orange')
axs_twin_test.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_test.tick_params(axis='y', labelcolor='orange')

fig.suptitle('Number of Layers in DBN', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.88)

plt.savefig('img/Nb_Layers.png')
plt.show()

In [ ]:
pretrained = to_study.query("`Pre-trained` == True and Layers == 2 and `Train data` == 60000")
without_pretrained = to_study.query("`Pre-trained` == False and Layers == 2 and `Train data` == 60000")

nb_neurons_dbn = pretrained["Neurons"].values

e_pretrained_dnn_train = pretrained["Error Train"].values
e_pretrained_dnn_test = pretrained["Error Test"].values
e_dnn_train = without_pretrained["Error Train"].values
e_dnn_test = without_pretrained["Error Test"].values

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].plot(nb_neurons_dbn, e_pretrained_dnn_train, label='Pre-trained DNN', marker='o', color='blue')
axs[0].set_xlabel('Number of Neurons in DBN')
axs[0].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[0].tick_params(axis='y', labelcolor='blue')
axs[0].set_title('Train')
axs[0].grid(True)

axs_twin_train = axs[0].twinx()
axs_twin_train.plot(nb_neurons_dbn, e_dnn_train, label='Standard DNN', marker='o', color='orange')
axs_twin_train.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_train.tick_params(axis='y', labelcolor='orange')

axs[1].plot(nb_neurons_dbn, e_pretrained_dnn_test, label='Pre-trained DNN', marker='o', color='blue')
axs[1].set_xlabel('Number of Neurons in DBN')
axs[1].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[1].tick_params(axis='y', labelcolor='blue')
axs[1].set_title('Test')
axs[1].grid(True)

axs_twin_test = axs[1].twinx()  
axs_twin_test.plot(nb_neurons_dbn, e_dnn_test, label='Standard DNN', marker='o', color='orange')
axs_twin_test.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_test.tick_params(axis='y', labelcolor='orange')

fig.suptitle('Number of Neurons in DBN', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.88)

plt.savefig('img/Nb_Neurons.png')
plt.show()

In [ ]:
pretrained = to_study.query("`Pre-trained` == True and Layers == 2 and Neurons == 200")
without_pretrained = to_study.query("`Pre-trained` == False and Layers == 2 and Neurons == 200")

nb_data = pretrained["Train data"].values

e_pretrained_dnn_train = pretrained["Error Train"].values
e_pretrained_dnn_test = pretrained["Error Test"].values
e_dnn_train = without_pretrained["Error Train"].values
e_dnn_test = without_pretrained["Error Test"].values

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].plot(nb_data, e_pretrained_dnn_train, label='Pre-trained DNN', marker='o', color='blue')
axs[0].set_xlabel('Number of Data')
axs[0].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[0].tick_params(axis='y', labelcolor='blue')
axs[0].set_title('Train')
axs[0].grid(True)

axs_twin_train = axs[0].twinx()
axs_twin_train.plot(nb_data, e_dnn_train, label='Standard DNN', marker='o', color='orange')
axs_twin_train.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_train.tick_params(axis='y', labelcolor='orange')

axs[1].plot(nb_data, e_pretrained_dnn_test, label='Pre-trained DNN', marker='o', color='blue')
axs[1].set_xlabel('Number of Data')
axs[1].set_ylabel('Pre-Trained - Error Rate (%)', color='blue')
axs[1].tick_params(axis='y', labelcolor='blue')
axs[1].set_title('Test')
axs[1].grid(True)

axs_twin_test = axs[1].twinx()  
axs_twin_test.plot(nb_data, e_dnn_test, label='Standard DNN', marker='o', color='orange')
axs_twin_test.set_ylabel('Standard - Error Rate (%)', color='orange')
axs_twin_test.tick_params(axis='y', labelcolor='orange')

fig.suptitle('Number of Data', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.88)

plt.savefig('img/Nb_Data.png')
plt.show()